In [1]:
from zernike_algos import (
    ZernikePolynomial,
    zernike_radial,
    zernike_radial_optimized,
)
import numpy as np

In [2]:
# Zernike radial direct computes the Zernike radial polynomials and respective derivatives
def zernike_radial_direct(r, n, m, dr):
    """Radial part of zernike polynomials. Modified Prata's algorithm."""
    
    m = abs(m) # m must be positive

    # Define a 3D matrix to save the radial polynomials R_n^m(rho)
    # and respective deritivatives d^l( R_n^m(rho) )/drho^l
    
    # First entries are the values of the polynomials at each value of the position array r
    # Second and third entries index the R_n^m polynomial.
    
    # The R array saves the evaluated polynomials according to the index convention n,m
    # The conventional output in DESC is a 2D list with the evaluated polynomials stacked
    
    R = np.full((r.size, max(n)+1, max(m)+1), np.nan)
    R_out = np.zeros(r.size)
    
    # Generate respective matrices if derivatives of R are to be computed
    if dr > 0:
        d1R = R  # d(R_nmk)/dr
    
    if dr > 1:
        d2R = R  # d2(R_nmk)/dr2
    
    if dr > 2:
        d3R = R  # d3(R_nmk)/dr3
    
    if dr > 3:
        d4R = R  # d4(R_nmk)/dr4

    # First step: Compute R_n^n using the simple relation R[n,n] = rho^n
    for i in (0, len(n)-1):
        
        n_i = n[i]
                
        # Compute 0th-derivative
        if n_i == 0:
            R[:, n_i, n_i] = np.ones(r.size)
        else:
            R[:, n_i, n_i] = r ** n_i      
        # Compute 1st-derivative
        if dr > 0:
            if n_i == 0:
                d1R[:, n_i, n_i] = np.zeros(r.size)
            if n_i == 1:
                d1R[:, n_i, n_i] = n_i * np.ones(r.size)
            if n_i > 1:
                d1R[:, n_i, n_i] = n_i * r ** (n_i - 1)       
        # Compute 2nd-derivative
        if dr > 1:
            if n_i <= 1:
                d2R[:, n_i, n_i] = np.zeros(r.size)
            if n_i == 2:
                d2R[:, n_i, n_i] = n_i*(n_i - 1) * np.ones(r.size)
            if n_i > 2:
                d2R[:, n_i, n_i] = n_i*(n_i - 1) * r ** (n_i - 2)        
        # Compute 3rd-derivative
        if dr > 2:
            if n_i <= 2:
                d3R[:, n_i, n_i] = np.zeros(r.size)
            if n_i == 3:
                d3R[:, n_i, n_i] = n_i*(n_i - 1)*(n_i - 2) * np.ones(r.size)
            if n_i >= 2:
                d3R[:, n_i, n_i] = n_i*(n_i - 1) * r ** (n_i - 2)
        # Compute 4th-derivative
        if dr > 3:
            if n_i <= 3:
                d4R[:, n_i, n_i] = np.zeros(r.size)
            if n_i == 4:
                d4R[:, n_i, n_i] = n_i*(n_i - 1)*(n_i - 2)*(n_i - 3) * np.ones(r.size)
            if n_i >= 3:
                d4R[:, n_i, n_i] = n_i*(n_i - 1)*(n_i - 2)*(n_i - 3) * r ** (n_i - 4)
    
    # Second step: Compute R[n,0] using the conventional relation for R_n^m(rho)
    for i in (0, len(n)-1):
        
        n_i = n[i]
        
        # Right now we are using the non-optimized zernike radial evaluation
        # When it's ready, we should switch to the optimized version
        if isinstance(n_i/2, int):
            
            R[:, n_i, 0] = zernike_radial(r, n_i, 0, 0)

            # Compute 1st-derivative
            if dr > 0:
                d1R[:, n_i, 0] = zernike_radial(r, n_i, 0, 1)
            # Compute 2nd-derivative
            if dr > 1:
                d2R[:, n_i, 0] = zernike_radial(r, n_i, 0, 2)
            # Compute 3rd-derivative
            if dr > 2:
                d3R[:, n_i, 0] = zernike_radial(r, n_i, 0, 3)       
            # Compute 4th-derivative
            if dr > 3:
                d4R[:, n_i, 0] = zernike_radial(r, n_i, 0, 4)
    
    # Third step: Compute the remaining R_n^m
    for i in (0, len(n)-1):
        
        n_i = n[i]

        for j in (0, len(m)-1):
        
            m_i = m[j]

            # (n_i + m_i)/2 Must be an integer
            if isinstance((n_i + m_i)/2, int):
            
                # There is a condition that dictates when to use the recursive or the direct relation
                if  n_i < m_i + 2:

                    R[:, n_i, m_i] = zernike_radial(r, n_i, m_i, 0)

                    # Compute 1st-derivative
                    if dr > 0:
                        d1R[:, n_i, m_i] = zernike_radial(r, n_i, m_i, 1)
                    # Compute 2nd-derivative
                    if dr > 1:
                        d2R[:, n_i, m_i] = zernike_radial(r, n_i, m_i, 2)
                    # Compute 3rd-derivative
                    if dr > 2:
                        d3R[:, n_i, m_i] = zernike_radial(r, n_i, m_i, 3)               
                    # Compute 4th-derivative
                    if dr > 3:
                        d4R[:, n_i, m_i] = zernike_radial(r, n_i, m_i, 4)

                elif m_i + 2 <= n_i < m_i:

                    K_1 = 2 * n_i / (m_i + n_i)
                    K_2 = 1 - K_1

                    # Compute 0th-derivative
                    R[:, n_i, m_i] = r * K_1 * R[:, n_i - 1, m_i - 1] + K_2 * R[:, n_i - 2, m_i]

                    # Compute 1st-derivative
                    if dr > 0:
                        # Recursive relation
                        d1R[:, n_i, m_i] = (K_1 * (R[:, n_i - 1, m_i - 1] + r * d1R[:, n_i - 1, m_i - 1])
                                            + K_2 * d1R[:, n_i - 2, m_i])
                    # Compute 2nd-derivative
                    if dr > 1:
                        # Recursive relation
                        d2R[:, n_i, m_i] = (K_1 * (2*d1R[:, n_i - 1, m_i - 1] 
                                                + r * d2R[:, n_i - 1, m_i - 1])
                                            + K_2 * d2R[:, n_i - 2, m_i])
                    # Compute 3rd-derivative
                    if dr > 2:
                        # Recursive relation
                        d3R[:, n_i, m_i] = (K_1 * (3*d2R[:, n_i - 1, m_i - 1] 
                                                + r * d3R[:, n_i - 1, m_i - 1])
                                            + K_2 * d3R[:, n_i - 2, m_i])
                    # Compute 4th-derivative
                    if dr > 3:
                        # Recursive relation
                        d4R[:, n_i, m_i] = (K_1 * (4*d3R[:, n_i - 1, m_i - 1] 
                                                + r * d4R[:, n_i - 1, m_i - 1])
                                            + K_2 * d4R[:, n_i - 2, m_i])
    
        # Define the output depending on the derivative desired
        if dr == 0:
            out = R
        if dr == 1:
            out = d1R
        if dr == 2:
            out = d2R
        if dr == 3:
            out = d3R
        if dr == 4:
            out = d4R

    return out

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[[      0.          0.         -0.    ...       0.         -0.
        0.   ]
 [      0.          0.         -0.    ...       0.         -0.
        0.   ]
 [      0.          0.         -0.    ...       0.         -0.
        0.   ]
 ...
 [      0.          0.         -0.    ...  493186.38   223447.05
    45063.434]
 [      0.          0.         -0.    ... 1221495.2    428588.47
    72975.74 ]
 [      0.          0.         -0.    ... 2717073.     794973.44
   117600.   ]]


zernike_radial, 0th derivative
1.32 ms ± 8.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
zernike_radial, 1st derivative
2.77 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
zernike_radial, 2nd derivative
3.94 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
zernike_radial, 3rd derivative
4.87 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
